In [23]:
class a:
    def __init__(self, idx) -> None:
        self.idx = idx
    def kill(self):
        if self.idx == 2:
            return True
        else:
            return False

l = [a(i) for i in range(4)]
[print(b.idx) for b in l]
l = [c for c in l if not c.kill()]
[print(b.idx) for b in l]

print('done')


0
1
2
3
0
1
3
done
